In [92]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import random


from sklearn import linear_model, datasets

In [93]:
def shrink(t,q):
    x = q[0]; y = q[1]
    dx = -x
    dy = -y
    return np.array([-x, -y])

In [96]:

num_trials = 10 # this is the number of intital conditions to try, note the total number of trials is num_trials*num_trials

min_x = -10
max_x = 10

min_y = -10
max_y = 10

x_vals = np.linspace(min_x, max_y, num_trials)
y_vals = np.linspace(min_y, max_y, num_trials)

dt = .1
t_eval = np.arange(0,5,dt)

q = np.zeros((len(t_eval), num_trials, num_trials, 2))

for i, x in enumerate(x_vals):
    for j, y in enumerate(y_vals):
        

        sol = solve_ivp(shrink, (0,5), np.array([x,y]), vectorized = True, t_eval = t_eval)
        q[:,i,j,:] = sol.y.T

        
        
        plt.plot(sol.y[0,:], sol.y[1,:])
        
        
%matplotlib inline
#plt.gca().set_aspect('equal', adjustable='box')
traj_list = [np.flip(q[:,i,j,:], axis = 0) for i in range(num_trials) for j in range(num_trials)]

In [73]:
# Calculate thefractional dimension at each time step (or something like that anyway)
# Method Katie Suggests

# TODO make this a function

# just calculating the length of the shortest trajectory
min_t = min([x.shape[0] for x in traj_list]) 

NR_list = []
r_delta = .01
delta = .01
for i in range(min_t):
    NR_list.append([])
    
    r_min = delta**2 # Length of the ball around our point. Delta from above, need to start somewhere
    r_max = 3
    num_r = 100
    N = 0
    
    for r in np.linspace(r_min, r_max, num=num_r):
        N = 0
        points = [traj[i,:] for traj in traj_list]
        random.shuffle(points) #shuffles points in place

        while(True):
            # Is a try catch really the best way to terminate?
            # probably not but right now I don't care
            try:
                center = points.pop(0) # pop also removes point from our list
                points[:] = [x for x in points if sum((center - x)**2) > r]
                N+=1
            except IndexError:
                NR_list[i].append((N,r))
                break
        

In [88]:
%matplotlib

a = np.array(NR_list[30])

plt.plot(a[:,1], a[:,0],'x-')
plt.figure()

plt.plot(np.log(a[:,1]), np.log(a[:,0]),'-')
#plt.figure()


X = np.log(a[:,1]).reshape(-1,1)
y = np.log(a[:,0]).reshape(-1,1)

# adapted from https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html

lr = linear_model.LinearRegression()
lr.fit(X, y)

ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Predict data of estimated models
line_X = np.arange(X.min(), X.max())[:, np.newaxis]
#line_y = lr.predict(line_X)
line_y_ransac = ransac.predict(line_X)

plt.scatter(X[inlier_mask], y[inlier_mask], color='yellowgreen', marker='.',
            label='Inliers')
plt.scatter(X[outlier_mask], y[outlier_mask], color='gold', marker='.',
            label='Outliers')

plt.plot(line_X, line_y_ransac)

Using matplotlib backend: MacOSX
